#  Segmenting and Clustering Neighborhoods in Toronto (Part 2)

This notebook is part of Applied Data Science Capstone course @Coursera.

In part 1, Postal data was scraped, engineered, having Latitude and Longitude added, and saved to CSV. Here we load the CSV back in to continue working on it.

In [12]:
import pandas as pd
df = pd.read_csv('Canada_Postal.csv', index_col=0)
print(df.head())
print(df.shape)

  PostalCode           Borough                                 Neighborhood  \
0        M3A        North York                                    Parkwoods   
1        M4A        North York                             Victoria Village   
2        M5A  Downtown Toronto                    Regent Park, Harbourfront   
3        M6A        North York             Lawrence Manor, Lawrence Heights   
4        M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government   

    Latitude  Longitude  
0  43.753259 -79.329656  
1  43.725882 -79.315572  
2  43.654260 -79.360636  
3  43.718518 -79.464763  
4  43.662301 -79.389494  
(103, 5)


## Exploring data set

We are going to visualize the neighborhood with map. 

In [13]:
from geopy.geocoders import Nominatim
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium

First we find the average of latitude/longtitude to use as center of our map.

In [22]:
latitude = df2.Latitude.mean()
longitude = df2.Longitude.mean()
print('The coordinates we are using are {}, {}.'.format(latitude, longitude))

The coordinates we are using are 43.704607733980595, -79.3971529116505.


A few test runs revealed zoom_start=11 to be the ideal number

In [23]:
map = folium.Map(location=[latitude, longitude], zoom_start=11)
for lat, lng, borough, neighborhood in zip(df2['Latitude'], df2['Longitude'], df2['Borough'], df2['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map)
    
map

Maps

In [10]:
df2.Borough.unique()

array(['North York', 'Downtown Toronto', 'Etobicoke', 'Scarborough',
       'East York', 'York', 'East Toronto', 'West Toronto',
       'Central Toronto', 'Mississauga'], dtype=object)